# Imports

In [65]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import os
from scipy.stats import norm
import sys
import pandas as pd
import pickle

## Helper Functions

In [66]:
%store -r analysis_hf
sys.path.insert(0, analysis_hf)

import covariance as c
import visualization as viz

# Set Directory Paths

In [67]:
# Directory path to load formatted Pathology Data
%store -r path_dataDir

# Directory path to save Calcualted Analysis Data
%store -r path_CalData

# [1] Load Formatted Pathology Data (TAU and TDP)

### Loading WM Pathology Data

In [68]:
pathT_WM = pd.read_csv(os.path.join(path_dataDir, 'new_pathT(WM).csv'))

In [69]:
pathT_WM

,INDDID,FullAutopsyID,AutopsyIDNumOnly,Tau1_TDP2,AnalysisRegion,ANG_L,ATC_L,HIP_L,IFC_L,M1_L,...,SPC_R,V1_R,aCING_R,aINS_R,aITC_R,dlPFC_R,iPFC_R,mPFC_R,pCING_R,pSTC_R
0,100071.0,2015-075,2015-075,2.0,WM,0.000313,NaN,NaN,NaN,0.012319,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,100551.0,2016-146,2016-146,1.0,WM,5.763064,NaN,6.363267,22.38985,3.067493,...,4.924144,NaN,NaN,NaN,NaN,12.000354,10.404543,NaN,NaN,NaN
2,100686.0,2009-097,2009-097,2.0,WM,0.003622,NaN,NaN,NaN,0.037712,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,101068.0,2004-267 UCSF,2004-267,1.0,WM,0.203100,NaN,0.100971,NaN,0.121270,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,101105.0,2002-159,2002-159,1.0,WM,0.793817,NaN,0.014950,NaN,0.436840,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174,118765.0,2014-092,2014-092,2.0,WM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.003521,0.002495,NaN,NaN,NaN,NaN,NaN,NaN,NaN
175,118935.0,2014-232,2014-232,2.0,WM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.001177,0.020103,NaN,NaN,NaN,NaN,NaN,NaN,NaN
176,119010.0,2014-256,2014-256,2.0,WM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.005002,0.000143,NaN,NaN,NaN,NaN,NaN,NaN,NaN
177,119052.0,2014-200,2014-200,2.0,WM,NaN,NaN,NaN,NaN,NaN,...,NaN,0.000000,0.000129,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Loading TAU Pathology Data

In [70]:
# path_TAU_WM
with open(os.path.join(path_dataDir, 'path_TAU_WM.pkl'), 'rb') as f:
    path_TAU_WM = pickle.load(f)
f.close()

In [71]:
print(path_TAU_WM.shape)

(76, 40)


### Loading TDP Pathology Data

In [72]:
# path_TDP_WM
with open(os.path.join(path_dataDir, 'path_TDP_WM.pkl'), 'rb') as f:
    path_TDP_WM = pickle.load(f)
f.close()

In [73]:
print(path_TDP_WM.shape)

(103, 40)


# [2] Parameter/Variable Setting

### P-value Threshold List

In [74]:
# pthresh_list = [0.05]
pthresh = 0.05

### Covariance Matrix Threshold (to remove noise)

In [75]:
cov_thresh = 0.1

### Label (Pathology Region) Names we are able to map to 3D. In alphabetical order (_L first and then _R)

In [76]:
LabelNames = pathT_WM.columns.values[5:]

In [77]:
len(LabelNames)

40

In [78]:
LabelNames

array(['ANG_L', 'ATC_L', 'HIP_L', 'IFC_L', 'M1_L', 'MFC_L', 'OFC_L',
       'PC_L', 'S1_L', 'SMTC_L', 'SPC_L', 'V1_L', 'aCING_L', 'aINS_L',
       'aITC_L', 'dlPFC_L', 'iPFC_L', 'mPFC_L', 'pCING_L', 'pSTC_L',
       'ANG_R', 'ATC_R', 'HIP_R', 'IFC_R', 'M1_R', 'MFC_R', 'OFC_R',
       'PC_R', 'S1_R', 'SMTC_R', 'SPC_R', 'V1_R', 'aCING_R', 'aINS_R',
       'aITC_R', 'dlPFC_R', 'iPFC_R', 'mPFC_R', 'pCING_R', 'pSTC_R'],
      dtype=object)

# [3] Calculate Covariance Matrices

## 1. TAU

In [79]:
path_TAU_WM.shape

(76, 40)

In [80]:
path_TDP_WM.shape

(103, 40)

In [81]:
covTAU_WM = c.covCal(path_TAU_WM, path_TAU_WM, cov_thresh)

## 2. TDP

In [82]:
covTDP_WM = c.covCal(path_TDP_WM, path_TDP_WM, cov_thresh)

## 3. TAU > TDP and TDP > TAU

In [83]:
covTAU_gt_TDP_WM, covTDP_gt_TAU_WM = c.covCalSigXY(path_TAU_WM, path_TDP_WM, covTAU_WM, covTDP_WM, pthresh, cov_thresh)

## 4. TAU > TDP and TDP > TAU RAW

In [84]:
covTAU_gt_TDP_raw_WM, covTDP_gt_TAU_raw_WM = c.covCalSigXYRaw(path_TAU_WM, path_TDP_WM, covTAU_WM, covTDP_WM, cov_thresh)

# Saving

### Save the Covariance Matrices

In [85]:
# covTAU_WM
with open(os.path.join(path_CalData, 'covTAU_WM.pkl'), 'wb') as f:
    pickle.dump(covTAU_WM, f)
f.close()

# covTDP_WM
with open(os.path.join(path_CalData, 'covTDP_WM.pkl'), 'wb') as f:
    pickle.dump(covTDP_WM, f)
f.close()

# covTAU_gt_TDP_WM
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_WM.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP_WM, f)
f.close()

# covTDP_gt_TAU_WM
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_WM.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU_WM, f)
f.close()

# covTAU_gt_TDP_raw_WM
with open(os.path.join(path_CalData, 'covTAU_gt_TDP_raw_WM.pkl'), 'wb') as f:
    pickle.dump(covTAU_gt_TDP_raw_WM, f)
f.close()

# covTDP_gt_TAU_raw_WM
with open(os.path.join(path_CalData, 'covTDP_gt_TAU_raw_WM.pkl'), 'wb') as f:
    pickle.dump(covTDP_gt_TAU_raw_WM, f)
f.close()

### Save LabelNames

In [86]:
# LabelNames
with open(os.path.join(path_CalData, 'LabelNames.pkl'), 'wb') as f:
    pickle.dump(LabelNames, f)
f.close()